In [22]:
import pandas as pd
import logging

# Set up logging for debugging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants
INPUT_CSV = "all_aircraft_flight_data_test12.csv"
OUTPUT_CSV = "silver_data_v14.csv"
DATETIME_COLUMNS = ["real_departure", "real_arrival"]
# Try multiple datetime formats
DATETIME_FORMATS = [
    "%Y-%m-%d %H:%M:%S",  # e.g., 2023-10-15 14:30:00
    "%m/%d/%Y %H:%M:%S",  # e.g., 10/15/2023 14:30:00
    "%Y-%m-%d %H:%M",     # e.g., 2023-10-15 14:30
    "%m/%d/%Y %H:%M"      # e.g., 10/15/2023 14:30
]

# Expected input columns to keep
KEEP_COLUMNS = [
    "tail_number",
    "aircraft_model",
    "real_departure",
    "real_arrival",
    "status",
    "icao_code",
    "airline_name",
    "destination_airport_lat",
    "destination_airport_lon"
]

def load_data(file_path: str) -> pd.DataFrame:
    """Load CSV and validate columns."""
    try:
        logging.info(f"Loading CSV: {file_path}")
        df = pd.read_csv(file_path)
        
        # Validate required columns
        missing_cols = [col for col in KEEP_COLUMNS if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")
        
        # Log sample data for debugging
        logging.info(f"Sample data for {DATETIME_COLUMNS}:\n{df[DATETIME_COLUMNS].head().to_string()}")
        return df
    except FileNotFoundError:
        raise FileNotFoundError(f"Input file not found: {file_path}")
    except Exception as e:
        raise Exception(f"Error loading CSV: {str(e)}")

def process_data(df: pd.DataFrame) -> pd.DataFrame:
    """Process DataFrame: filter columns, convert datetimes, compute durations, transform status."""
    # Keep only desired columns
    logging.info("Filtering columns")
    df = df[KEEP_COLUMNS].copy()
    
    # Convert datetime columns with multiple format attempts
    logging.info("Converting datetime columns")
    for col in DATETIME_COLUMNS:
        # Store original values for debugging
        original_values = df[col].copy()
        df[col] = pd.to_datetime(df[col], errors="coerce")  # Initial flexible parse
        # Try specific formats if all NaT or many failures
        if df[col].isna().sum() > len(df) * 0.5:  # If more than 50% are NaT
            logging.warning(f"High NaT rate in {col}, trying specific formats")
            for fmt in DATETIME_FORMATS:
                try:
                    temp = pd.to_datetime(original_values, format=fmt, errors="coerce")
                    if temp.isna().sum() < df[col].isna().sum():
                        df[col] = temp
                        logging.info(f"Format {fmt} improved parsing for {col}")
                except Exception as e:
                    logging.debug(f"Format {fmt} failed for {col}: {str(e)}")
        
        # Log invalid values for debugging
        invalid_rows = df[col].isna()
        if invalid_rows.any():
            logging.warning(f"Invalid datetime values in {col} (count: {invalid_rows.sum()}):")
            logging.warning(f"Sample invalid values:\n{original_values[invalid_rows].head().to_string()}")
    
    # Flag rows with valid datetimes for processing
    df["valid_datetime"] = ~df[DATETIME_COLUMNS].isna().any(axis=1)
    logging.info(f"Rows with valid datetimes: {df['valid_datetime'].sum()} out of {len(df)}")
    
    # Extract datetime components (only for valid rows)
    logging.info("Extracting datetime components")
    df = df.assign(
        departure_month=lambda x: x["real_departure"].dt.month.where(x["valid_datetime"]),
        departure_day=lambda x: x["real_departure"].dt.day.where(x["valid_datetime"]),
        departure_year=lambda x: x["real_departure"].dt.year.where(x["valid_datetime"]),
        departure_date_str=lambda x: x["real_departure"].dt.strftime("%m/%d/%Y").where(x["valid_datetime"])
    )
    
    # Log missing datetime components
    for col in ["departure_month", "departure_day", "departure_year", "departure_date_str"]:
        missing_count = df[col].isna().sum()
        if missing_count > 0:
            logging.warning(f"Missing values in {col}: {missing_count}")
    
    # Compute duration (only for valid rows)
    logging.info("Computing duration")
    df = df.assign(
        duration=lambda x: (x["real_arrival"] - x["real_departure"]).where(x["valid_datetime"]),
        duration_minutes=lambda x: (x["duration"].dt.total_seconds() / 60).where(x["valid_datetime"]),
        duration_hours=lambda x: (x["duration"].dt.total_seconds() / 3600).where(x["valid_datetime"])
    )
    
    # Log missing duration values
    for col in ["duration", "duration_minutes", "duration_hours"]:
        missing_count = df[col].isna().sum()
        if missing_count > 0:
            logging.warning(f"Missing values in {col}: {missing_count}")
    
    # Transform status: 'landed' to 1, others to 0, handle 'estimated'
    logging.info("Transforming status column")
    df["status"] = df["status"].str.lower().replace({"landed": 1}).fillna("unknown")
    df["status"] = pd.to_numeric(df["status"], errors="coerce").fillna(0).astype(int)
    
    # Filter out 'estimated' status (optional, commented out)
    # logging.info("Filtering out 'estimated' status")
    # df = df[~df["status"].astype(str).str.contains("estimated", na=False)]
    
    # Add record_id and cycle (assuming cycle is a flight counter)
    logging.info("Adding record_id and cycle")
    df["record_id"] = range(1, len(df) + 1)
    df["cycle"] = 1  # Simplistic; adjust based on actual cycle logic (e.g., group by tail_number)
    
    return df

def save_data(df: pd.DataFrame, file_path: str) -> None:
    """Save DataFrame to CSV."""
    logging.info(f"Saving DataFrame to {file_path}")
    try:
        df.to_csv(file_path, index=False)
        logging.info(f"Data saved to {file_path}")
    except Exception as e:
        raise Exception(f"Error saving CSV: {str(e)}")

def main():
    """Main function to process flight data."""
    try:
        # Load data
        df = load_data(INPUT_CSV)
        
        # Process data
        final_df = process_data(df)
        
        # Drop unnecessary columns
        logging.info("Dropping unnecessary columns")
        final_df = final_df.drop(["real_arrival", "duration", "valid_datetime"], axis=1, errors="ignore")
        
#         # Rename columns
#         logging.info("Renaming columns")
#         expected_columns = [
#             "tail_number",
#             "aircraft_model",
#             "real_departure",
#             "status",
#             "icao_code",
#             "airline_name",
#             "destination_airport_lat",
#             "destination_airport_lon",
#             "month",
#             "day",
#             "year",
#             "date",
#             "duration_min",
#             "duration_hr",
#             "cycle"
#         ]
#         if len(final_df.columns) != len(expected_columns):
#             logging.error(f"Column count mismatch: expected {len(expected_columns)} ({expected_columns}), got {len(final_df.columns)} ({list(final_df.columns)})")
#             raise ValueError("Column count mismatch")
        
#         final_df.columns = expected_columns
        
        # Save and display results
        save_data(final_df, OUTPUT_CSV)
        logging.info("Processed DataFrame (first 5 rows):")
        print(final_df.head())
        
    except Exception as e:
        logging.error(f"Error in main: {str(e)}")
        raise

if __name__ == "__main__":
    main()

2025-05-14 15:50:34,431 - INFO - Loading CSV: all_aircraft_flight_data_test12.csv
2025-05-14 15:50:34,435 - INFO - Sample data for ['real_departure', 'real_arrival']:
    real_departure     real_arrival
0              NaN              NaN
1              NaN              NaN
2  5/14/2025 19:35              NaN
3   5/14/2025 1:27  5/14/2025 16:48
4   5/13/2025 8:38  5/13/2025 20:48
2025-05-14 15:50:34,436 - INFO - Filtering columns
2025-05-14 15:50:34,437 - INFO - Converting datetime columns
2025-05-14 15:50:34,458 - WARNING - Invalid datetime values in real_departure (count: 37):
2025-05-14 15:50:34,459 - WARNING - Sample invalid values:
0     NaN
1     NaN
14    NaN
15    NaN
18    NaN
2025-05-14 15:50:34,480 - WARNING - Invalid datetime values in real_arrival (count: 46):
2025-05-14 15:50:34,481 - WARNING - Sample invalid values:
0     NaN
1     NaN
2     NaN
14    NaN
15    NaN
2025-05-14 15:50:34,482 - INFO - Rows with valid datetimes: 253 out of 299
2025-05-14 15:50:34,483 - INFO -

  tail_number aircraft_model      real_departure  status icao_code  \
0      N23983           B789                 NaT       0       UAL   
1      N23983           B789                 NaT       0       UAL   
2      N23983           B789 2025-05-14 19:35:00       0       UAL   
3      N23983           B789 2025-05-14 01:27:00       1       UAL   
4      N23983           B789 2025-05-13 08:38:00       1       UAL   

      airline_name  destination_airport_lat  destination_airport_lon  \
0  United Airlines                28.566500                77.103081   
1  United Airlines                40.692501               -74.168602   
2  United Airlines                50.037796                 8.555783   
3  United Airlines                28.566500                77.103081   
4  United Airlines                40.692501               -74.168602   

   departure_month  departure_day  departure_year departure_date_str  \
0              NaN            NaN             NaN                NaN   
1 